In [3]:
import requests
from bs4 import BeautifulSoup
from typing import List, Dict, Optional
import pandas as pd
from pydantic import BaseModel
from selenium import webdriver


In [46]:
## Ingredientai
resp = requests.get("https://www.15min.lt/maistas/receptas/varskes-ir-fermentinio-surio-pyragas-8796")
soup = BeautifulSoup(resp.content, "html.parser")
all_ingredients: List[Dict] = []

recipe_title = soup.find("div", class_="recipe-head").find("h1").text.strip()
recipe_image = soup.find("div", class_="image").find("img").get("src")
recipe_ingredients = soup.find("ul", class_="ingredients").find_all("li")
for ingredient in recipe_ingredients:
    the_ingredient = ingredient.find_all("span")
    all_ingredients.append({
        "ingredient_name": the_ingredient[0].text.strip(),
        "ingredient_amount": the_ingredient[2].text.strip(),
        "ingredient_note": the_ingredient[3].text.strip()
    })

#all_ingredients

## Paruosimo budas
recipe_manual: List[str] = []
recipe_making_steps = soup.find("div", class_="description text").find_all("p")

recipe_making_steps



for step in recipe_making_steps:
    stepsas = step.text.strip()
    recipe_manual.append(stepsas)

In [35]:
## Info apie viena recepta

def extract_recipe_ingredients(url: str) -> str:
    ## Ingredientai
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    all_ingredients: List[Dict] = []

    recipe_title = soup.find("div", class_="recipe-head").find("h1").text.strip()
    recipe_image = soup.find("div", class_="image").find("img").get("src")
    recipe_ingredients = soup.find("ul", class_="ingredients").find_all("li")
    for ingredient in recipe_ingredients:
        ingredient_span = ingredient.find_all("span")
        all_ingredients.append({            
                "ingredient_name": ingredient_span[0].text.strip(),
                "ingredient_amount": ingredient_span[2].text.strip(),
                "ingredient_note": ingredient_span[3].text.strip()
            })
       
    return str(all_ingredients)


def extract_making_steps(url: str) -> str:

    ## Paruosimo budas
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    recipe_manual: List[str] = []
    recipe_making_steps = soup.find("div", class_="description text").find_all("p")

    for step in recipe_making_steps:
        step_to_txt = step.text.strip()
        recipe_manual.append(step_to_txt)
    
    return "\n".join(recipe_manual) 


In [10]:
## IKEA search products

# sudeda paieskos rezultatos is pirmo psl

def search_products(url) -> List[Dict]:
    """Function for doing search in ikea.lt"""
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    products: List[Dict] = []

    search_results_all = soup.find_all("div", class_="col-6 col-md-4 col-lg-3 p-0 itemBlock")
    for product in search_results_all:
        link = product.find("div", class_="itemInfo").a.get("href")
        products.append({            
                    "product_name": product.find("div", class_="itemInfo").a.text.strip(),
                    "product_info": product.find("div", class_="itemInfo").find("div", class_ = "itemPrice-wrapper").span.text.replace(" €", ""),
                    "product_link": f"https://www.ikea.lt{link}",
                    "product_image": product.find("div", class_ = "productImg").img.get("src")
                })

    return products
    
#prod = search_products("https://www.ikea.lt/lt/search/?q=kede")
#prod

In [4]:
## IKEA retrieve links

from pydantic import BaseModel

__items_per_page__: int = 40
__domain__: str = "https://www.ikea.lt"

class FurnitureLink(BaseModel):
    url: str


class Furniture(BaseModel):
    """Class for base properties of furniture"""
    furniture_name: str
    furniture_description: str
    furniture_price = int
    furniture_image_link: List[str]    
    furniture_key_features: str
    furniture_size: str
    
    furniture_stock_in_store: int


def _get_page_content(query: str) -> Optional[BeautifulSoup]:
        """Method to get needed content from search result page or whatever related page."""
        resp = requests.get(query)
        if resp.status_code == 200:
            return BeautifulSoup(resp.content)
        raise Exception("Cannot get content. Site is unreachable.")



#################################################

def _retrieve_item_links(results_count: int, keyword: str) -> List[str]:  #[FurnitureLink]: # prideti self
    """Method to search furnitures by keyword and save specifed number of results."""
    results: List[str] = []  #[FurnitureLink] = []        
    pages_to_iterate: int = (results_count // __items_per_page__) + 1 # kiek paieskos rezultatu puslapiu naudoti        
    
    for page_num in range(1, pages_to_iterate + 1):
        content = _get_page_content(f"{__domain__}/lt/search/?q={keyword}&page={page_num}") #pries metoda prideti self
        pages_count = content.find("ul", class_="pagination mb-0")
        no_results = content.find("div", class_="col mt-4")
        
        if pages_count != None:
            max_number_of_pages = int(content.find("ul", class_="pagination mb-0").find_all("li", class_="page-item")[3].text)
        elif no_results != None:           
            max_number_of_pages = 0
        else:
            max_number_of_pages = 1
        
        if content:
            if max_number_of_pages >= pages_to_iterate:                        
                all_items_per_page = content.find_all("div", class_="col-6 col-md-4 col-lg-3 p-0 itemBlock")
                counter = 1
                
                while results_count >= counter:                                                                    
                    for item in all_items_per_page:
                        link = item.find("div", class_="itemInfo").a.get("href")
                        item = f"{__domain__}{link}"
                        results.append(item)  #(FurnitureLink(url = item))
                        counter += 1                        
                    
            else: 
                print(f"Search has less results than requested.")
                break
        if not content:
            break

    return results
    

In [35]:
test = _retrieve_item_links(1, "ledai")
print(test)




['https://www.ikea.lt/lt/products/virtuve/maisto-ruosimas-ir-produktu-laikymas/virtuviniai-itaisai/chosigt-ledu-formeles-ivairiu-spalvu-art-80208478', 'https://www.ikea.lt/lt/products/virtuve/maisto-ruosimas-ir-produktu-laikymas/virtuviniai-itaisai/chosigt-ledu-kauselis-melyna-rausva-geltona-zalia-art-90208246', 'https://www.ikea.lt/lt/products/virtuve/maisto-ruosimas-ir-produktu-laikymas/virtuviniai-itaisai/utspadd-ledu-formeles-melyna-rausva-art-40308352']


In [11]:
## IKEA get key features

def _extract_key_features(content: BeautifulSoup) -> str:  # pridet self
    results: List[str] = []   
    features = content.find("div", class_="tab-pane_box").find_all("p")
    
    for feature in features: 
        feature_line = feature.text.strip()
        results.append(feature_line)       
        
    return results

In [122]:
def _extract_furniture_size(content: BeautifulSoup) -> str: #pridet 
    results: List[str] = []
    measures = content.find_all("div", class_="tab-pane_box")[1].find_all("tr") #.find("table")
    
    for measure in measures:
        measure_line = measure.text.strip()
        results.append(measure_line)
    return results

In [7]:
## IKEA extract main info
#resp = requests.get("https://www.ikea.lt/lt/products/valgomasis/sedimieji-valgomojo-baldai/kedes-ir-suoliukai/terje-sulankstomoji-kede-sviesi-pilka-balta-art-80456982")
#resp = requests.get("https://www.ikea.lt/lt/products/ikea-verslui/nekilnojamasis-turtas/sofos/jattebo-dviviete-moduline-sofa-tamsi-geltona-zalia-spr-29471405")
resp = requests.get("https://www.ikea.lt/lt/products/miegamasis/sistema-pax/pakabai-kiti-drabuziu-avalynes-priedai/omtanksam-batu-saukstas-antracito-sp-art-70378070")
content = BeautifulSoup(resp.content, "html.parser")



##############################


def _retrieve_furniture_info(link: str) -> Optional[Furniture]: #prideti self, link: FurnitureLink prie attr
        """Method to get main info about furniture."""
        content = _get_page_content(link.url)  # prideti self prie get metodo

        if content:
            try:
                furniture_name = content.find("div", class_="d-flex align-items-center flex-wrap").h3.text
                furniture_description = content.find("h4", class_="itemFacts font-weight-normal").span.text
                furniture_price = int(content.find("div", class_="itemPrice-wrapper").p.span.text.replace(" €", ""))
            except AttributeError:
                return None

            try:
                furniture_image_link = content.find("a", class_ = "slideImg").find("img").get("src")
            except KeyError:
                furniture_image_link = None

            return Furniture(
                furniture_name = furniture_name,
                furniture_description = furniture_description,
                furniture_price = furniture_price,
                furniture_image_link = furniture_image_link,
                furniture_key_features = _extract_key_features(content), # self
                furniture_size = _extract_furniture_size(content), # self

                #furniture_stock_in_store = int,                
            )
        else:
            return None

In [ ]:
## pabandyt su selenium, nes reikia hitint mygtuka

from selenium.webdriver.common.by import By

def _check_number_in_stock(content: BeautifulSoup) -> str: # self
    quantity = content.find("div", class_="message noCursor d-flex").find("p").text
    driver = webdriver.Chrome(options=options,chrome_options=chrome_options)
    driver.implicitly_wait(10)
    driver.get(url)
    #time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

In [149]:
## SELENIUM
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service # del netinkamos driverio versijos galima autoamtiskai sutvarkjyt
#from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By


# run Selenium in the background by configuring webdriver
options = webdriver.ChromeOptions()
options.headless = True # hide GUI
options.add_argument("start-maximized") # open gui in a full screen mode
options.add_experimental_option("excludeSwitches", ["enable_automation"])
options.add_experimental_option("detach", True)


# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    # this will disable image loading
    "prefs", {"profile.managed_default_content_settings.images": 2}
)


#url = "https://www.nordpoolgroup.com/en/Market-data1/Dayahead/Area-Prices/LT/Hourly/?view=table"
url = "https://www.ikea.lt/lt/products/ikea-verslui/nekilnojamasis-turtas/sofos/soderhamn-2-viete-sofa-su-gulimuoju-kreslu-tamsiai-pilka-spr-39449616"
driver = webdriver.Chrome(options=options,chrome_options=chrome_options)
#driver.implicitly_wait(10)
driver.get(url)
#time.sleep(5)
button = driver.find_element(By.CLASS_NAME, "special-tooltip mb-0" )
button.click()

print()


#BeautifulSoup(page_source, 'html.parser')

#page_source = driver.page_source
#soup = BeautifulSoup(page_source, 'html.parser')


# nepamirst driver.close() or driver.quit()





C:\Users\laimis\AppData\Local\Temp\ipykernel_10196\2355329164.py:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options,chrome_options=chrome_options)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".special-tooltip mb-0"}
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x008CACD3+2075859]
	Ordinal0 [0x0085EE61+1633889]
	Ordinal0 [0x0075B7BD+571325]
	Ordinal0 [0x0078AC2F+764975]
	Ordinal0 [0x0078AE1B+765467]
	Ordinal0 [0x007BD0F2+970994]
	Ordinal0 [0x007A7364+881508]
	Ordinal0 [0x007BB56A+963946]
	Ordinal0 [0x007A7136+880950]
	Ordinal0 [0x0077FEFD+720637]
	Ordinal0 [0x00780F3F+724799]
	GetHandleVerifier [0x00B7EED2+2769538]
	GetHandleVerifier [0x00B70D95+2711877]
	GetHandleVerifier [0x0095A03A+521194]
	GetHandleVerifier [0x00958DA0+516432]
	Ordinal0 [0x0086682C+1665068]
	Ordinal0 [0x0086B128+1683752]
	Ordinal0 [0x0086B215+1683989]
	Ordinal0 [0x00876484+1729668]
	BaseThreadInitThunk [0x7674FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77467BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77467B8E+238]


In [144]:
print(soup)

<html><!--<![endif]--><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide{display:none !important;}ng\:form{display:block;}.ng-animate-block-transitions{transition:0s all!important;-webkit-transition:0s all!important;}.ng-hide-add-active,.ng-hide-remove{display:block!important;}</style>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=0" name="viewport"/>
<title>Market data | Nord Pool</title>
<meta content="Market data" name="DC.title"/>
<meta content="https://www.nordpoolgroup.com/en/Market-data1/Dayahead/Area-Prices/LT/Hourly/" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="Market data" property="og:title"/>
<meta content="Nord Pool runs the leading power market in Europe, and we offer day-ahead and intraday markets to our customers. 

Trade power in 16 countries and add related services such as compliance, data or courses.  " property="og:d

In [12]:
feat = _extract_key_features(content)
feat

''